In [1]:
import os, glob, math
import numpy as np

In [228]:
import enterprise
import enterprise.pulsar as ep
import enterprise.signals.parameter as esp
import enterprise.signals.signal_base as esb
import enterprise.signals.white_signals as esw

from enterprise.signals.signal_base import function, Function

In [ ]:
os.environ['TEMPO2'] = '/Users/vallis/miniconda3/envs/sksparse/share/tempo2'

## Enterprise `Parameter`

In [3]:
x = esp.Uniform(0,1)

In [15]:
x

enterprise.signals.parameter.Uniform.<locals>.Uniform

In [5]:
x.get_logpdf(0.5)

TypeError: You can only call get_logpdf() on an
                            instantiated (named) Parameter.

In [6]:
x1 = x('x1')

In [16]:
x1

x1:Uniform(pmin=0, pmax=1)

In [7]:
x1.get_logpdf(0.5)

0.0

## Enterprise `Function` (positional + keywork argument)

In [146]:
def add(a, x=1, y=2):
    return a + x + y

f = Function(add, x=x)

In [147]:
f1 = f('f1')

In [148]:
f1

f1(f1_x:Uniform(pmin=0, pmax=1))

In [149]:
f1.params

[f1_x:Uniform(pmin=0, pmax=1)]

In [150]:
f1(2)

5

In [151]:
f1(2,0.5,7)

9.5

This is OK to fail, but seems the wrong error message.

In [152]:
f1(params={'x': 1})

TypeError: add() missing 1 required positional argument: 'a'

This should work?

In [153]:
f1(params={'a': 1})

TypeError: add() missing 1 required positional argument: 'a'

Note you cannot assign y directly!

In [103]:
f1(2,x=0.5,y=7)

4.5

In [97]:
f1(2,params={"f1_x": 1.5})

5.5

## `Function` with `Function` parameter 

In [159]:
def doub(x):
    return 2*x

f = Function(doub, x=esp.Uniform(0,1))

def mult(a, w=2, z=1):
    return a * w * z

g = Function(mult, w=esp.Uniform(2,3), z=f)

In [160]:
g1 = g('g1')

In [161]:
g1

g1(g1_w:Uniform(pmin=2, pmax=3), g1_z_x:Uniform(pmin=0, pmax=1))

In [162]:
g1.params

[g1_w:Uniform(pmin=2, pmax=3), g1_z_x:Uniform(pmin=0, pmax=1)]

Does not use the z default... messy to fix because it would have to know that it does not have enough information to compute the function before calling it.

In [163]:
g1(2)

TypeError: doub() missing 1 required positional argument: 'x'

These work.

In [168]:
g1(2,z=3)

12

In [165]:
g1(2,w=10,z=3)

60

This also works. Should it? Potentially problematic if there are parameter name collisions...

In [169]:
g1(2,x=5)

40

In [171]:
g1(2,params={'g1_z_x': 5, 'g1_w': 10})

200

## Justin's use case

In [173]:
def powerlaw(f, log10_A=-15):
    return (10**log10_A) * f**2

pl = Function(powerlaw, log10_A=esp.Uniform(0,5))

In [183]:
pl1 = pl('pl1')

In [184]:
pl1

pl1(pl1_log10_A:Uniform(pmin=0, pmax=5))

In [204]:
fs = np.array([1,2,3])

In [205]:
pl1(fs)

array([  1.00000000e-15,   4.00000000e-15,   9.00000000e-15])

In [206]:
pl1(fs, log10_A=-16)

array([  1.00000000e-16,   4.00000000e-16,   9.00000000e-16])

Note the name of the parameter is not just log10_A. Is this the desired default behavior?

In [207]:
pl1(fs, params={'pl1_log10_A': -17})

array([  1.00000000e-17,   4.00000000e-17,   9.00000000e-17])

In [199]:
def log10(A=10**-16):
    return math.log10(A)

pm = Function(powerlaw, log10_A=Function(log10,A=esp.Uniform(10**-17,10**-14)))

In [200]:
pm1 = pm('pm1')

In [201]:
pm1

pm1(pm1_log10_A_A:Uniform(pmin=1e-17, pmax=1e-14))

In [208]:
pm1(fs, log10_A=-13)

array([  1.00000000e-13,   4.00000000e-13,   9.00000000e-13])

Note this uses the log10 default, not the powerlaw default...

In [209]:
pm1(fs)

array([  1.00000000e-16,   4.00000000e-16,   9.00000000e-16])

Finally this is what Justin cares about. The naming of the parameter is somewhat abstruse... 

In [210]:
pm1(fs, params={'pm1_log10_A_A': 10**-19})

array([  1.00000000e-19,   4.00000000e-19,   9.00000000e-19])

But it doesn't work (yet) with `Signal` parameters.

In [229]:
datadir = 'tests/data'

parfiles = sorted(glob.glob(datadir + '/mdc1/*.par'))
timfiles = sorted(glob.glob(datadir + '/mdc1/*.tim'))

psr = enterprise.pulsar.Pulsar(parfiles[0], timfiles[0])

In [250]:
equad = esw.EquadNoise(log10_equad=esp.Normal(0,10))

In [251]:
equad1 = equad(psr)

In [252]:
equad1.params

[J0030+0451_log10_equad:Normal(mu=0, sigma=10)]

Something wrong here... the caching?

In [267]:
equad1.get_ndiag(params={'J0030+0451_log10_equad:': -13})[:10]

ndarray_alt([  1.00000000e-16,   1.00000000e-16,   1.00000000e-16,
         1.00000000e-16,   1.00000000e-16,   1.00000000e-16,
         1.00000000e-16,   1.00000000e-16,   1.00000000e-16,
         1.00000000e-16])

The underlying function seems fine.

In [262]:
equad1._ndiag[''].params

[J0030+0451_log10_equad:Normal(mu=0, sigma=10)]

In [266]:
equad1._ndiag[''](params={'J0030+0451_log10_equad': -13})[:10]

array([  1.00000000e-26,   1.00000000e-26,   1.00000000e-26,
         1.00000000e-26,   1.00000000e-26,   1.00000000e-26,
         1.00000000e-26,   1.00000000e-26,   1.00000000e-26,
         1.00000000e-26])

Nevertheless, let's try with a `Function` parameter

In [268]:
fquad = esw.EquadNoise(log10_equad=Function(log10,A=esp.Uniform(10**-17,10**-14)))

In [269]:
fquad1 = fquad(psr)

In [286]:
fquad1.params

[J0030+0451_log10_equad_A:Uniform(pmin=1e-17, pmax=1e-14)]

In [272]:
fquad1._ndiag[''].params

[J0030+0451_log10_equad_A:Uniform(pmin=1e-17, pmax=1e-14)]

In [282]:
fquad1._ndiag[''](params={'J0030+0451_log10_equad_A': 10**-14})[:10]

array([  1.00000000e-28,   1.00000000e-28,   1.00000000e-28,
         1.00000000e-28,   1.00000000e-28,   1.00000000e-28,
         1.00000000e-28,   1.00000000e-28,   1.00000000e-28,
         1.00000000e-28])

Hmm... this does not work either... although it uses the default of log10.

In [289]:
fquad1.get_ndiag(params={'J0030+0451_log10_equad_A:': -13})[:10]

ndarray_alt([  1.00000000e-32,   1.00000000e-32,   1.00000000e-32,
         1.00000000e-32,   1.00000000e-32,   1.00000000e-32,
         1.00000000e-32,   1.00000000e-32,   1.00000000e-32,
         1.00000000e-32])

## Earlier tests, disregard...

In [5]:
@function
def gaussianprior(x, x0=0, std=1):
    return np.exp(-0.5 * (x - x0)**2 / std**2) / np.sqrt(2 * math.pi * std**2)

In [24]:
@esb.function
def pl(f,log10_A=2):
    return f**(2*log10_A)

In [30]:
@esb.function
def log10(A):
    return np.log10(A)

In [51]:
F = pl(log10_A=log10(A=esp.Uniform(0,1)))

In [54]:
iF = F('F')

In [55]:
iF.params

[]

In [7]:
a = esp.Uniform(0,1)

In [8]:
a1 = a('uniform')

In [9]:
a1

"uniform":Uniform(0,1)

In [10]:
a1.params

["uniform":Uniform(0,1)]

In [15]:
a1.get_pdf(0.5)

1.0

In [11]:
b = esp.UserParameter(gaussianprior(x0=esp.Uniform(-1,1), std=esp.Uniform(-2,2)))

In [12]:
b1 = b('gauss')

In [13]:
b1

"gauss":UserParameter(gauss_std,gauss_x0)

In [14]:
b1.params

["gauss":UserParameter(gauss_std,gauss_x0),
 "gauss_std":Uniform(-2,2),
 "gauss_x0":Uniform(-1,1)]

In [16]:
b1.get_pdf(0.5)

0.35206532676429952

In [17]:
b1.get_pdf(params={'gauss': 0.5, 'gauss_x0': 0, 'gauss_std': 1})

0.35206532676429952

In [18]:
b1.get_pdf(params={'gauss': 0.5, 'gauss_x0': 1, 'gauss_std': 2})

0.19333405840142465

In [19]:
def f(x, a=0):
    return x + a

In [21]:
c = esb.Function(f,a = esp.UserParameter(gaussianprior(x0=esp.Uniform(-1,1), std=esp.Uniform(-2,2))))

In [22]:
c1 = c('secondary')

In [23]:
c1.params

["secondary_a":UserParameter(secondary_a_std,secondary_a_x0),
 "secondary_a_std":Uniform(-2,2),
 "secondary_a_x0":Uniform(-1,1)]

In [30]:
c1(0,params={'secondary_a': 2})

2